In [68]:
# Run if using tensorflow2.0+
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Add, Activation, InputLayer
from tensorflow.keras import Input
from tensorflow.keras.optimizers import SGD, Adam

In [ ]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Add, Activation
from keras import Input

In [2]:
df_train = pd.read_csv('../Data/train.csv')
df_train['open_channels'].value_counts(normalize=True)

0     0.248030
1     0.197173
3     0.133722
2     0.110785
4     0.080682
5     0.055575
7     0.053003
8     0.049037
6     0.037622
9     0.027224
10    0.007147
Name: open_channels, dtype: float64

In [42]:
def get_data(batch_num='all', zeros = True):
    #create training data per batch
    batch_size = 500000
    if batch_num == 'all':
        beg = 0
        end = 499999999
        print('Training on all data')
    else:
        batch = batch_num
        beg = (batch-1)*batch_size 
        end = batch*batch_size - 1
    df_batch = df_train[beg:end]
    
    # filter out the zero open_channels cases
    if not zeros:
        df_batch = df_batch.drop(df_batch[df_batch['open_channels']==0].index) #optional, added because model was always predicting 0
        
    # randomize 
    signal = np.array(df_batch.signal)
    open_channels = np.array(df_batch.open_channels)
    x_train, x_test, y_train, y_test = train_test_split(signal,open_channels,test_size=0.25)
    x_train = np.reshape(x_train, (-1,1))
    y_train = np.reshape(y_train, (-1,1))
    x_test = np.reshape(x_test, (-1,1))
    y_test = np.reshape(y_test, (-1,1))
    
    # categorize outputs
    enc = OneHotEncoder()
    enc.fit_transform(y_train)
    enc.fit_transform(y_test)
    
    return x_train, x_test, y_train, y_test

In [27]:
x_train,x_test,y_train,y_test = get_data()
enc = OneHotEncoder()
enc.fit(y_train)
enc.categories_;
enc.transform(y_train)
y_train

Training on all data


array([[3],
       [3],
       [0],
       ...,
       [4],
       [3],
       [0]], dtype=int64)

In [48]:
def resnet(layer_size=64, dropout=0, depth=1):
    #model params
    layer = layer_size
    dropout = dropout
    layers = depth
    
    #create model
    inputs = Input(shape=(1,))

    if depth > 0:
        k = Dense(layer, activation='relu')(inputs)
        k = Dense(layer, activation='relu')(k)
        k = Dropout(dropout)(k)
        block1 = Add()([k, inputs])
        kl = Dense(20, activation='relu')(block1)
        
        if depth > 1:
            k = Dense(layer, activation='relu')(block1)
            k = Dense(layer, activation='relu')(k)
            k = Dropout(dropout)(k)
            block2 = Add()([k, block1])
            kl = Dense(20, activation='relu')(block2)

            if depth > 2:
                k = Dense(layer, activation='relu')(block2)
                k = Dense(layer, activation='relu')(k)
                k = Dropout(dropout)(k)
                block3 = Add()([k, block2])
                kl = Dense(20, activation='relu')(block3)

                if depth > 3:
                    k = Dense(layer, activation='relu')(block3)
                    k = Dense(layer, activation='relu')(k)
                    k = Dropout(dropout)(k)
                    block4 = Add()([k, block3])
                    kl = Dense(20, activation='relu')(block4)
                    
                    if depth > 4:
                        k = Dense(layer, activation='relu')(block4)
                        k = Dense(layer, activation='relu')(k)
                        k = Dropout(dropout)(k)
                        block5 = Add()([k, block4])
                        kl = Dense(20, activation='relu')(block5)
                        
                        if depth > 5:
                            k = Dense(layer, activation='relu')(block5)
                            k = Dense(layer, activation='relu')(k)
                            k = Dropout(dropout)(k)
                            block6 = Add()([k, block5])
                            kl = Dense(20, activation='relu')(block6)
                        
                            if depth > 6:
                                k = Dense(layer, activation='relu')(block6)
                                k = Dense(layer, activation='relu')(k)
                                k = Dropout(dropout)(k)
                                block7 = Add()([k, block6])
                                kl = Dense(20, activation='relu')(block7)
                                
                                if depth > 7:
                                    k = Dense(layer, activation='relu')(block7)
                                    k = Dense(layer, activation='relu')(k)
                                    k = Dropout(dropout)(k)
                                    block8 = Add()([k, block7])
                                    kl = Dense(20, activation='relu')(block8)

    outputs = Dense(units=11, activation='softmax')(kl)
    
    model = Model(inputs,outputs)
    return model

In [50]:
x_train,x_test,y_train,y_test = get_data(1,zeros=True)
model = resnet(layer_size=256, dropout=0.1, depth=8)
model.compile(loss='sparse_categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, validation_data=(x_test,y_test), batch_size=100, verbose=1)

Train on 374999 samples, validate on 125000 samples
Epoch 1/5
374999/374999 [==============================] - 42s 111us/sample - loss: 0.0890 - accuracy: 0.9682 - val_loss: 0.0838 - val_accuracy: 0.9677
Epoch 2/5
374999/374999 [==============================] - 46s 122us/sample - loss: 0.0716 - accuracy: 0.9691 - val_loss: 0.0479 - val_accuracy: 0.9749
Epoch 3/5
374999/374999 [==============================] - 45s 121us/sample - loss: 0.0385 - accuracy: 0.9865 - val_loss: 0.0079 - val_accuracy: 0.9977
Epoch 4/5
374999/374999 [==============================] - 45s 121us/sample - loss: 0.0137 - accuracy: 0.9958 - val_loss: 0.0163 - val_accuracy: 0.9948
Epoch 5/5
374999/374999 [==============================] - 46s 121us/sample - loss: 0.0094 - accuracy: 0.9969 - val_loss: 0.0300 - val_accuracy: 0.9892


In [58]:
acc = []
for batch in range(1,11):    
    x_train,x_test,y_train,y_test = get_data(batch,zeros=True)
    model = resnet(layer_size=256, dropout=0.1, depth=8)
    model.compile(loss='sparse_categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=10, validation_data=(x_test,y_test), batch_size=100, verbose=1)
    
    y_pred = model.predict(x_test)
    score = accuracy_score(y_test, y_pred)
    acc.append(score)
acc

Train on 374999 samples, validate on 125000 samples
Epoch 1/10
374999/374999 [==============================] - 41s 109us/sample - loss: 0.0867 - accuracy: 0.9682 - val_loss: 0.0716 - val_accuracy: 0.9683
Epoch 2/10
374999/374999 [==============================] - 50s 133us/sample - loss: 0.0497 - accuracy: 0.9808 - val_loss: 0.0117 - val_accuracy: 0.9980
Epoch 3/10
374999/374999 [==============================] - 45s 120us/sample - loss: 0.0152 - accuracy: 0.9954 - val_loss: 0.0082 - val_accuracy: 0.9974
Epoch 4/10
374999/374999 [==============================] - 47s 125us/sample - loss: 0.0095 - accuracy: 0.9968 - val_loss: 0.0061 - val_accuracy: 0.9978
Epoch 5/10
374999/374999 [==============================] - 47s 125us/sample - loss: 0.0082 - accuracy: 0.9972 - val_loss: 0.0090 - val_accuracy: 0.9968
Epoch 6/10
374999/374999 [==============================] - 48s 127us/sample - loss: 0.0078 - accuracy: 0.9974 - val_loss: 0.0060 - val_accuracy: 0.9979
Epoch 7/10
374999/374999 [====

ValueError: Classification metrics can't handle a mix of binary and continuous-multioutput targets

In [ ]:
x_train,x_test,y_train,y_test = get_data(zeros=True)
model = resnet(layer_size=256, dropout=0.1, depth=8)
model.compile(loss='sparse_categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, validation_data=(x_test,y_test), batch_size=100, verbose=1)

In [70]:
# Create model for flat signal type (batches 1-6)
x_tr1,x_t1,y_tr1,y_t1 = get_data(1)
x_tr2,x_t2,y_tr2,y_t2 = get_data(2)
x_tr3,x_t3,y_tr3,y_t3 = get_data(3)
x_tr4,x_t4,y_tr4,y_t4 = get_data(4)
x_tr5,x_t5,y_tr5,y_t5 = get_data(5)
x_tr6,x_t6,y_tr6,y_t6 = get_data(6)
x_train = np.concatenate((x_tr1,x_tr2,x_tr3,x_tr4,x_tr5,x_tr6))
x_test = np.concatenate((x_t1,x_t2,x_t3,x_t4,x_t5,x_t6))
y_train = np.concatenate((y_tr1,y_tr2,y_tr3,y_tr4,y_tr5,y_tr6))
y_test = np.concatenate((y_t1,y_t2,y_t3,y_t4,y_t5,y_t6))

model1 = resnet(layer_size=175, dropout=0.1, depth=4)
opt = Adam(learning_rate=.01,beta_1=0.95)
model1.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
model1.fit(x_train, y_train, epochs=6, validation_data=(x_test,y_test), batch_size=80, verbose=1)

Train on 2249994 samples, validate on 750000 samples
Epoch 1/6
2249994/2249994 [==============================] - 148s 66us/sample - loss: 0.4876 - accuracy: 0.8271 - val_loss: 0.4126 - val_accuracy: 0.8458
Epoch 2/6
2249994/2249994 [==============================] - 148s 66us/sample - loss: 0.3900 - accuracy: 0.8576 - val_loss: 0.3777 - val_accuracy: 0.8623
Epoch 3/6
2249994/2249994 [==============================] - 153s 68us/sample - loss: 0.3849 - accuracy: 0.8594 - val_loss: 0.3939 - val_accuracy: 0.8567
Epoch 4/6
2249994/2249994 [==============================] - 155s 69us/sample - loss: 0.3827 - accuracy: 0.8602 - val_loss: 0.3762 - val_accuracy: 0.8608
Epoch 5/6
2249994/2249994 [==============================] - 156s 69us/sample - loss: 0.3813 - accuracy: 0.8605 - val_loss: 0.3817 - val_accuracy: 0.8599
Epoch 6/6
2249994/2249994 [==============================] - 156s 69us/sample - loss: 0.3803 - accuracy: 0.8609 - val_loss: 0.3765 - val_accuracy: 0.8624


In [71]:
model1.save('../Models/resnet_flat_85.h5')

In [65]:
# Create model for parabolic signal type (batches 2,7-10)
x_tr2,x_t2,y_tr2,y_t2 = get_data(2)
x_tr7,x_t7,y_tr7,y_t7 = get_data(7)
x_tr8,x_t8,y_tr8,y_t8 = get_data(8)
x_tr9,x_t9,y_tr9,y_t9 = get_data(9)
x_tr10,x_t10,y_tr10,y_t10 = get_data(10)
x_train = np.concatenate((x_tr2,x_tr7,x_tr8,x_tr9,x_tr10))
x_test = np.concatenate((x_t2,x_t7,x_t8,x_t9,x_t10))
y_train = np.concatenate((y_tr2,y_tr7,y_tr8,y_tr9,y_tr10))
y_test = np.concatenate((y_t2,y_t7,y_t8,y_t9,y_t10))

model = resnet(layer_size=300, dropout=0.1, depth=2)
model.compile(loss='sparse_categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])
model.fit(x_train, y_train, epochs=6, validation_data=(x_test,y_test), batch_size=80, verbose=1)

Train on 1874995 samples, validate on 625000 samples
Epoch 1/6
1874995/1874995 [==============================] - 157s 84us/sample - loss: 1.6745 - accuracy: 0.3803 - val_loss: 1.4651 - val_accuracy: 0.4478
Epoch 2/6
1874995/1874995 [==============================] - 175s 93us/sample - loss: 1.3833 - accuracy: 0.4683 - val_loss: 1.3156 - val_accuracy: 0.4924
Epoch 3/6
1874995/1874995 [==============================] - 175s 94us/sample - loss: 1.3213 - accuracy: 0.4926 - val_loss: 1.3251 - val_accuracy: 0.4837
Epoch 4/6
1874995/1874995 [==============================] - 176s 94us/sample - loss: 1.3018 - accuracy: 0.4981 - val_loss: 1.3012 - val_accuracy: 0.5036
Epoch 5/6
1874995/1874995 [==============================] - 175s 93us/sample - loss: 1.2947 - accuracy: 0.5000 - val_loss: 1.2994 - val_accuracy: 0.4953
Epoch 6/6
1874995/1874995 [==============================] - 174s 93us/sample - loss: 1.2897 - accuracy: 0.5010 - val_loss: 1.2936 - val_accuracy: 0.4951


In [72]:
model.save('../Models/resnet_curvy_49.h5')